In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/course-topics/Coursetopics.csv


In [2]:
%matplotlib inline

from pathlib import Path

import heapq
from collections import defaultdict
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [3]:
course = pd.read_csv('../input/course-topics/Coursetopics.csv')

In [4]:
course.head(11)

,Intro,DataMining,Survey,Cat Data,Regression,Forecast,DOE,SW
0,1,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,1,0,1,1,0,0,1
3,1,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0
7,0,0,0,1,0,1,1,1
8,1,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0


In [5]:
course['SW'].unique()

array([0, 1])

In [6]:
course.isnull().sum()

Intro         0
DataMining    0
Survey        0
Cat Data      0
Regression    0
Forecast      0
DOE           0
SW            0
dtype: int64

#use association rules to analyze these data and interpret several of the resulting rules

In [7]:
len(course)

365

In [8]:
itemsets = apriori(course, min_support=12/365, use_colnames=True)

# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)
rules.sort_values(by=['lift'], ascending=False).head(6)

print(rules.sort_values(by=['lift'], ascending=False)
      .drop(columns=['antecedent support', 'consequent support', 'conviction'])
      .head(6))

              antecedents consequents   support  confidence      lift  \
1         (Intro, Survey)        (SW)  0.032877    0.545455  2.457912   
4     (Regression, Intro)        (SW)  0.038356    0.538462  2.426401   
3        (Regression, SW)     (Intro)  0.038356    0.700000  1.774306   
0            (SW, Survey)     (Intro)  0.032877    0.666667  1.689815   
2  (Regression, Cat Data)     (Intro)  0.032877    0.600000  1.520833   

   leverage  
1  0.019501  
4  0.022548  
3  0.016739  
0  0.013421  
2  0.011259  


Now suppose that we want association rules between items for this database that have a support count of at least 0.03 (equivalent to a percentage support of 12/365 = 3.3%). In other words, rules based on items that were purchased together in at least 3.3% of the transactions. By enumeration, we can see that only the itemsets listed above have a count of at least 0.03. 

If Intro and Survey are purchased, then with confidence 54.5% SW will also be purchased. This rule has a lift ratio of 2.46

If Intro and Regression are purchased, then with confidence 53.8% SW will also be purchased. This rule has a lift ratio of 2.43

If SW and Regression are purchased, then with confidence 70% Intro will also be purchased. This rule has a lift ratio of 1.77

If Survey and SW are purchased, then with confidence 66.7% Intro will also be purchased. This rule has a lift ratio of 1.69

If Cat Data and Regression are purchased, then with confidence 60% Intro will also be purchased. This rule has a lift ratio of 1.52

The minimum support for the database is 14/365 and after which no record will be captured.